In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# Lista de tuplas contendo (nome do clube, URL) dos times da Série A do Brasileirão 2024 no Transfermarkt
club_urls = [
    ('flamengo','https://www.transfermarkt.com.br/flamengo-rio-de-janeiro/startseite/verein/614'),
    ('corinthians','https://www.transfermarkt.com.br/corinthians-sao-paulo/startseite/verein/199'),
    ('bahia','https://www.transfermarkt.com.br/esporte-clube-bahia/startseite/verein/10010'),
    ('botafogo','https://www.transfermarkt.com.br/botafogo-rio-de-janeiro/startseite/verein/537'),
    ('sao-paulo','https://www.transfermarkt.com.br/fc-sao-paulo/startseite/verein/585'),
    ('athletico-pr','https://www.transfermarkt.com.br/club-athletico-paranaense/startseite/verein/679'),
    ('bragantino','https://www.transfermarkt.com.br/red-bull-bragantino/startseite/verein/8793'),
    ('palmeiras','https://www.transfermarkt.com.br/se-palmeiras-sao-paulo/startseite/verein/1023'),
    ('internacional','https://www.transfermarkt.com.br/sc-internacional-porto-alegre/startseite/verein/6600'),
    ('cruzeiro','https://www.transfermarkt.com.br/cruzeiro-ec/kader/verein/609/saison_id/2023'),
    ('atletico-mg','https://www.transfermarkt.com.br/clube-atletico-mineiro/startseite/verein/330'),
    ('fortaleza','https://www.transfermarkt.com.br/fortaleza-esporte-clube/startseite/verein/10870'),
    ('gremio','https://www.transfermarkt.com.br/gremio-porto-alegre/startseite/verein/210'),
    ('vasco','https://www.transfermarkt.com.br/vasco-da-gama-rio-de-janeiro/startseite/verein/978'),
    ('juventude','https://www.transfermarkt.com.br/esporte-clube-juventude/startseite/verein/10492'),
    ('fluminense','https://www.transfermarkt.com.br/fluminense-rio-de-janeiro/startseite/verein/2462'),
    ('criciuma','https://www.transfermarkt.com.br/criciuma-esporte-clube/startseite/verein/7178'),
    ('atletico-go','https://www.transfermarkt.com.br/atletico-clube-goianiense/startseite/verein/15172'),
    ('vitoria','https://www.transfermarkt.com.br/esporte-clube-vitoria/startseite/verein/2125'),
    ('cuiaba','https://www.transfermarkt.com.br/cuiaba-ec-mt-/startseite/verein/28022')
  ]

# Defina o cabeçalho para a solicitação
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

# Lista para armazenar os dados de todos os jogadores
all_player_data = []

# Itere sobre a lista de (nome do clube, URL)
for club_name, url in club_urls:
    # Faça a solicitação HTTP para a página
    response = requests.get(url, headers=headers)

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        print(f'Página acessada com sucesso: {url}')
        content = response.content
        soup = BeautifulSoup(content, 'html.parser')

        # Encontre a tabela de jogadores
        table = soup.find('table', {'class': 'items'})

        # Verifique se a tabela foi encontrada
        if table:
            print('Tabela encontrada!')
            # Obtenha todas as linhas da tabela
            rows = table.find_all('tr', {'class': ['odd', 'even']})

            # Itere sobre as linhas da tabela e extraia os dados dos jogadores
            for row in rows:
                # Nome do jogador
                player_name = row.find('td', {'class': 'hauptlink'}).text.strip()

                # Posição do jogador
                position = row.find_all('td')[4].text.strip()

                # Idade do jogador
                age = row.find_all('td')[5].text.strip()

                # Valor de mercado do jogador
                market_value = row.find('td', {'class': 'rechts hauptlink'}).text.strip()

                # Adicione os dados do jogador à lista
                all_player_data.append([player_name, club_name, position, age, market_value])
        else:
            print('Tabela não encontrada!')
    else:
        print(f'Erro ao acessar a página: {response.status_code}')

# Crie um DataFrame com os dados de todos os jogadores
transfermarket = pd.DataFrame(all_player_data, columns=['Nome do Jogador', 'Clube', 'Posição', 'Idade', 'Valor de Mercado'])

# Salve os dados em um arquivo Excel
path = 'dados_transfermarkt_serie_a_2024.xlsx'
transfermarket.to_excel(path, index=False)
print("Os dados foram salvos com sucesso em", path)

# Função para extrair a idade
def extrair_idade(valor):
    try:
        return valor.split(' ')[1].strip('()')  # Extrai a idade entre parênteses
    except IndexError:
        return None  # Retorna None se o formato for inválido

# Função para extrair o ano de nascimento
def extrair_ano_nascimento(valor):
    try:
        return valor.split(' ')[0]  # Extrai a data de nascimento
    except IndexError:
        return None  # Retorna None se o formato for inválido

# Aplicando as funções no DataFrame
transfermarket['Ano de Nascimento'] = transfermarket['Idade'].apply(extrair_ano_nascimento)
transfermarket['Idade'] = transfermarket['Idade'].apply(extrair_idade)

reais = 6.16

# Supondo que a coluna 'Valor de Mercado' contenha os valores em formato de string com o símbolo do euro
transfermarket['Valor de Mercado'] = transfermarket['Valor de Mercado'].str.replace('€', '').str.replace('mi.', '0000').str.replace('K', '0').str.replace(',', '').str.replace(' ', '')
transfermarket['Valor de Mercado'] = pd.to_numeric(transfermarket['Valor de Mercado'], errors='coerce')

# Agora, podemos multiplicar os valores da coluna 'Valor de Mercado' pelo valor em reais
transfermarket['Valor em Reais'] = transfermarket['Valor de Mercado'] * reais



# Obter a taxa de câmbio EUR/BRL
#exchange_rate = get_exchange_rate(api_key)

transfermarket.rename( columns={'Clube':'Equipe','Nome do Jogador':'Jogador'}, inplace=True)

transfermarket.to_excel("transfermarket.xlsx", index=False, engine='openpyxl')

Página acessada com sucesso: https://www.transfermarkt.com.br/flamengo-rio-de-janeiro/startseite/verein/614
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/corinthians-sao-paulo/startseite/verein/199
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/esporte-clube-bahia/startseite/verein/10010
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/botafogo-rio-de-janeiro/startseite/verein/537
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/fc-sao-paulo/startseite/verein/585
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/club-athletico-paranaense/startseite/verein/679
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/red-bull-bragantino/startseite/verein/8793
Tabela encontrada!
Página acessada com sucesso: https://www.transfermarkt.com.br/se-palmeiras-sao-paulo/startseite/verein/1023
Tabela encontrada!


C:\Users\bruno\AppData\Local\Temp\ipykernel_44240\3593667045.py:108: FutureWarning: The default value of regex will change from True to False in a future version.
  transfermarket['Valor de Mercado'] = transfermarket['Valor de Mercado'].str.replace('€', '').str.replace('mi.', '0000').str.replace('K', '0').str.replace(',', '').str.replace(' ', '')
